# Overview

* [Introduction](#Introduction)
    * Neo4j & Python
* [Setup](#Setup) 
    * A Note on Python Versions
    * Install Neo4j
    * Setup Database Authentication
    * Authentication's No Fun :(
    * Installing Python to Neo4j Middleware
* [Cypher Basics & Middleware Options](#Cypher-Basics-&-Middleware-Options)
    * Connecting to Neo4j & Creating a Session
    * The Cypher Query Language
    * A Basic API Example
    * API-Based Programmatic Access
* [Data Access](#Data-Access)
    * Just a Subset Please
    * Full Disease Module Detection DREAM Challenge Data   
* [Exploring The DREAM Data With NetworkX](#Exploring-The-Dream-Data-With-NetworkX)
* [Common Issues & Their Solutions](#Common-Issues-&-Their-Solutions)
    * If You Previously Installed Neo4j and Forgot Your Username/Password
    * Port Bind Exception
    * Server Certificate Does Not Match Known Certificate for 'localhost'
    * General SSL Engine Problem

# Introduction

This tutorial is written for people with a general technical background, specifically a basic knowledge of databases, working in the terminal, and a strong understanding of at least one computer language. It was written with Unix operating systems in mind, so while we discuss installation steps for Windows in brief and have made an effort to support it on a basic level, some of the code (i.e. most if not all the terminal commands) and file locations may differ. 

So, what can this tutorial do for you?

* Help you get a Neo4j database up and running quickly on your local machine
* Give you a quick guided tour of Neo4j & Cypher to see how they work together
* Expose you to two difference flavors of Neo4j to Python middleware
* Help you perform a preliminary analysis on a subset of data from the [Disease Module Identification DREAM Challenge](https://www.synapse.org/#!Synapse:syn6156761/wiki/400645)

That said, if you are already well-versed in Neo4j and feel comfortable with Cypher feel free to [grab the DREAM data](#Data-Access) and skip ahead to the analysis part of the tutorial. Also, don't forget to [install NetworkX](#Installing-Python-to-Neo4j-Middleware-&-NetworkX) for the analysis portion. We're excited to see what analyses people may run now and in the future. 

And of course, if you run into any issues, give a shout to either Ben, Eric, or Ian and we'll come lend a hand. Finally, we hope you find this a fun and approachable way to getting started with Neo4j. Happy keyboard mashing!

# Setup


### A Note on Python Versions
This tutorial was written using Python 3.5, so if you only have Python 2.x, you may find some things are different. While there are several (very small) differences between the two versions of the language, the ones that will be of interest for this tutorial relate to the print statement / function and Python's pacakge manager Pip:

 Python 2.x         | Python 3.x        
 ------------------ | :------------------:
 `print "oink"`     | `print("oink")`   
 `pip install woof` | `pip3 install woof`
 
 **<span style="color: red;">Caveat:</span>** *It is possible that if you only have a Python3 installation, you will not need to use `pip3` in place of `pip`.*

### Install Neo4j

We've tested two ways to setup Neo4j on Mac OS X and one on Microsoft Windows that you can choose from below. 

#### Homebrew – Mac OS X

There are a number of ways you can install Neo4j. If you have homebrew, type the following command into your terminal:
```
brew install neo4j
```
Be sure to take note of what version is installed (there's a chance you may need this in a moment); for me it was 3.0.3, but at the time of writing the latest is 3.0.6. 

#### Desktop Application – Mac OS X & Microsoft Windows

* Go to the [Neo4j website](https://neo4j.com)
* Click the big red Download button
* Click the green Download Community Edition button
* Select the appropriate download and run through the installation steps required by your OS

### Setup Database Authentication

It's recommended you setup a username/password for Neo4j (that said, read the caveat below before you do.) To do this:

* Go to your terminal application and open up a new window or pane, and type: 
```
neo4j console
```
* Next, in your browser navigate to [`http://localhost:7474`](http:////localhost:7474)
* You should be prompted to set a new username/password. Of course, make sure it's something you'll remember; resetting your user/pass in Neo4j tends to be a bit of terminal hackery and is poorly documented. If you're having trouble, you can skip down to the [Authentication's No Fun :(](#Authentication's-No-Fun-:-() section to disable authentication; if you already setup a user/pass and forgot it, check out [Some Helpful Installation Hints](#Some-Helpful-Installation-Hints).

**<span style="color: green;">Tip:</span>** *Once you've set this up, you may want to take a moment and explore the Neo4j web application. It's a cute way to play around with some toy databases, check out some nifty network visualizations, send some queries, and get a feel for the syntax and functionality of the Cypher query language. If you're thinking, 'Meh, I'm not into toys and cute stuff, I'm a hardcore bioinformaticist.', scroll down to the [Data Access](#Data-Access) portion for some much-less-cute (but really it's not bad at all) bioinformatics data.*

**<span style="color: red;">Caveat:</span>** *The user/pass you set will need to be hardcoded into the Python code of this tutorial early on, so if you feel more comfortable going with the default user/pass, follow the instructions above up to setting your user/pass then head down to [Data Access](#Data-Access)*

### Authentication's No Fun : (

Security through authentication is important, but we're realists and know that this is a tutorial...so if you installed with homebrew and are having trouble getting the authentication up and running quickly, you can shut off authentication. To do that, do the following:

* Use your terminal to navigate to the following directory (replace `VERSION` with the version number you just wrote down):
```
cd ~/../../usr/local/Cellar/neo4j/VERSION/libexec/conf/
```
* Now in the ```conf/``` directory, open the file ```neo4j.conf``` using your preferred terminal editor (e.g. Vim, Emacs, Nano)
* Scroll through the file until you find the following line (it should be near the top):
```
# dbms.security.auth_enabled=false
```
* Per the instructions above it, uncomment the line. It should look like this: 
```
dbms.security.auth_enabled=false
```
* Save and close the file `neo4j.conf`
* Pat your self on the back; your database is now authentication free...

**<span style="color: green;">Tip:</span>** *If you didn't install with Homebrew or are using a different OS (not Mac OS X), these steps should be the same but the location of the `neo4j.conf` file will be different.* 

**<span style="color: red;">Caveat:</span>** *With great power comes great responsibility. In the interest of getting started quickly, we're offering the above instructions as an option. That said, we feel obliged to say that disabling authentication should never, ever be done in a production environment or for any database(s) hosting sensitive data. If you plan on using Neo4j outside the scope of this tutorial, be sure to setup authentication as intended (above). That said, rest assured the Neo4j documentation states that, unless configured to do so, Neo4j will not respond to external requests (i.e. not from your localhost)...although, the same documentation also states the above may make you vulnerable to a number of known attacks (e.g. cross-site scripting).*

### Installing Python to Neo4j Middleware & NetworkX

Python does not natively support interacting with Neo4j and requires middleware to do so. There are several available each with their own flavor, one of which is officially sponsored by Neo4j. We'll be using two Python to Neo4j middlewares in this tutorial. 

The first is the [official Neo4j middleware](https://neo4j.com/developer/python/), which has been designed to expose the Cypher query language in the Python environment and facilitate connecting to any Neo4j instance.

The second middleware is the [unofficial Py2Neo](http://py2neo.org/v3/). The design of this middleware is different; rather than use the Cypher query language (i.e. bumble through learning a language with a syntax that may be quite foreign), Py2Neo provides us with a Pythonic API to access and query the database with almost no knowledge of the Cypher language required. Similar to the official Neo4j, it too allows for easy connecting to any Neo4j instance. 

To install the two middlewares, paste the commands below into your terminal:
```
pip3 install neo4j-driver
pip3 install py2neo
```

Finally, if you plan on exploring the [DREAM Disease Module Identification Challenge](https://www.synapse.org/#!Synapse:syn6156761) data, a great package to work with it is [NetworkX](https://networkx.github.io). To install it, paste the command below into you terminal:
```
pip3 install networkx
```

# Cypher Basics & Middleware Options

### Connecting to Neo4j & Creating a Session

In [353]:
from neo4j.v1 import GraphDatabase
from neo4j.v1 import basic_auth     as Authentication

# Connect to Neo4j
gdb     = GraphDatabase.driver("bolt://localhost",  auth = Authentication("neo4j", "test"))

# Create a Session
session = gdb.session()

**<span style="color: green;">Tip:</span>** *You may have noticed above that we're not connecting to the database with a common protocol (e.g. http, https, ssh). While Neo4j can make use of http and https, the developers of the software have also written an optimized protocol known as bolt.*

### The Cypher Query Language

You should now have access to your Neo4j instance through the active session. At this point, if you haven't already, it's worth running through some basic Cypher queries to get a feel for the language, what's happening under the hood, and how Neo4j thinks about data. 

As mentioned earlier, the Neo4j official driver (middleware) for Python simply exposes the Cypher query language through the `session.run()` method. There are benefits and drawbacks to this:

**<span style="color: green;">Some benefits:</span>**
* You get the full flexibility of the Cypher query language
* For running queries, you need one method only
* The driver provides access to database settings and configurations that other middlewares may not expose
* Officially supported & well documented

**<span style="color: red;">Some drawbacks:</span>**
* Requires a working knowledge of the Cypher query language
* Less Pythonic queries by default
* May require more hardcoding or the creation of a custom set of Python to Cypher functions

For those who are comfortable programmin in Python, the ability of the official driver to run Cypher queries directly makes it a great starting point. 

####  Deleting Nodes & Edges

Before jumping in, it's worth laying out how to delete nodes and edges so that you can hit the reset button if any queries aren't working as expected. 

In [354]:
# # To Delete All Relationships (Must Be Done Before Deleting Nodes)
# session.run("START n = node(*) MATCH n-[r]-() DELETE r")

# # To Delete All Nodes
# session.run("START n = node(*) MATCH n DELETE n")

# To Delete All Nodes & Relationships Simultaneously
session.run("MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n, r")

#### Creating & Querying Nodes

In [355]:
# Create Five Nodes
createNodes = "CREATE \
(Arthur:    Person {fullName: 'Artorius Pendragon',   title: 'King',    age: 37,  influence: 1.00, lovesSpam: true}), \
(Guinevere: Person {fullName: 'Gwythyr ap Greidawl',  title: 'Queen',   age: 32,  influence: 0.75, lovesSpam: true}), \
(Lancelot:  Person {fullName: 'Lancelot du Lac',      title: 'Knight',  age: 29,  influence: 0.25, lovesSpam: false}), \
(Merlin:    Person {fullName: 'Myrddin Wyllt',        title: 'Wizard',  age: 68,  influence: 0.50, lovesSpam: true})"
session.run(createNodes)

# Query The Four Generated Nodes
matchNodes  = "MATCH (Person) RETURN Person.fullName AS name, Person.title AS title, Person.lovesSpam AS spam"
records     = session.run(matchNodes)

# Print Results
print("A Legendary (kinda) Haiku of Yore\n")
for record in records:
    if record["spam"]:
        print("%s was a legendary %s that loved spam" % (record["name"], record["title"].lower()))
    else:
        print("Little %s was a legendary %s that had a very testy relationship with spam" % (record["name"], record["title"].lower()))

A Legendary (kinda) Haiku of Yore

Artorius Pendragon was a legendary king that loved spam
Gwythyr ap Greidawl was a legendary queen that loved spam
Little Lancelot du Lac was a legendary knight that had a very testy relationship with spam
Myrddin Wyllt was a legendary wizard that loved spam


#### Creating & Querying Edges

In [356]:
# Set This to a Relationship of Interest
queryRelationship = "SUBJECT_TO"

# Generate Some Edges - Note Arrows Can go Both Directions (see ADVISEE_TO relationship)
createEdges = "CREATE \
(Arthur)-[:HUSBAND_TO]->(Guinevere), \
(Arthur)-[:KING_TO]->(Lancelot), \
(Arthur)-[:KING_TO]->(Merlin), \
(Guinevere)-[:WIFE_TO]->(Arthur), \
(Guinevere)-[:FRIEND_TO]->(Lancelot), \
(Guinevere)-[:FRIEND_TO]->(Merlin), \
(Lancelot)-[:SUBJECT_TO]->(Arthur), \
(Lancelot)-[:FRIEND_TO]->(Guinevere), \
(Merlin)-[:ADVISOR_TO]->(Arthur), \
(Merlin)<-[:ADVISEE_TO]-(Arthur), \
(Merlin)-[:FRIEND_TO]->(Guinevere)"

session.run(createEdges)

# Query Path Between Nodes & Edge
queryEdges = "MATCH p = (a)-[r:" + queryRelationship + "]->(b) return p, r, a, b"
records = session.run(queryEdges)

# Print Results
for record in records:
    print("\n".join("%s: %s" % (key, record[key]) for key in record.keys()), "\n")

p: <Path start=1276 end=1274 size=1>
r: <Relationship id=1601 start=1276 end=1274 type='SUBJECT_TO' properties={}>
a: <Node id=1276 labels=set() properties={}>
b: <Node id=1274 labels=set() properties={}> 



#### Querying the Graph

If you're interested in learning more about Cypher and querying the Neo4j graph, it's worth cracking open the [Cypher documentation](http://neo4j.com/docs/2.2.8/cypher-query-lang.html) and writing some queries to answer some questions. For the graph above, some basic questions we could ask are:

* Who is the oldest character?
* Who loves spam?
* Who is the least influential?
* Who is a wizard?

But things could be much more interesting (particularly with real data!) For example, is there an association between a Person's influence and love of Spam? A question for the ages to be sure. There are numerous resources online that describe basic Cypher queries, here's a small selection we've found useful:

* [A Cypher Query Cheatsheet](https://gist.github.com/DaniSancas/1d5265fc159a95ff457b940fc5046887)
* [A Great DZone Blog Post Comparing Cypher to SQL](https://dzone.com/articles/rdbms-graphs-sql-vs-cypher-query-languages)
* [Some Nifty Neo4j & Cypher YouTube Tutorials](https://www.youtube.com/watch?v=UJ81zWBMguc&list=PLAWPhrZnH759YHRieMBzsQRvr56JcYx5l)

### A Basic API

If you're a believer in the benefits of intelligent abstraction, you've probably noticed that the above is not ideal (particularly if you're new to Cypher and its ASCII art syntax.) One solution may be to develop your own simple API to work with Cypher. If anything, such an approach can be a great way to learn the language and get acquainted with the Neo4j driver middleware. Below is an example of two methods that might compose part of such an API, though they are no doubt of poor quality and should never be used for any serious work. 

In [456]:
def createNode (id, attributes, session = session):
    queryAttr = ", ".join(["%s: '%s'" % (key, value) for key, value in attributes.items()])
    query     = "CREATE (n:%s {%s})" % (id, queryAttr)
    result    = session.run(query)
    return result, query

# Call the createNode Method
result, query = createNode('Person', {"title": 'king', "fullName": 'Artorius Pendragon', "age": 37, "influence": 1.00, "lovesSpam": True})
print(query)
print(result, "\n")

def createEdge (nodeA, relationship, nodeB, session = session):
    query     = "CREATE (%s)-[:%s]->(%s)" % (nodeA, relationship, nodeB)
    result    = session.run(query)
    return result, query

# Call the createEdge Method
result, query = createEdge('Merlin', "ADVISOR_TO", "Arthur")
print(query)
print(result)

CREATE (n:Person {title: 'king', age: '37', influence: '1.0', lovesSpam: 'True', fullName: 'Artorius Pendragon'})

CREATE (Merlin)-[:ADVISOR_TO]->(Arthur)


# API-Based Programmatic Access

As mentioned earlier, sometimes you want to skip learning a new language like Cypher. The Py2Neo middleware, for the most part, allows for this convenience, however, as with the official driver there are benefits & drawbacks (below). Next, we'll run through a similar set of queries to demonstrate some very basic Py2Neo functionalities.

**<span style="color: green;">Some benefits:</span>**
* You get the full flexibility of the Cypher query language (yes, with Py2Neo, you can still use it if required!)
* The syntax is largely Pythonic, minimizing language context switching while programming
* The driver provides access to database settings and configurations that other middlewares may not expose
* Less hardcoding & more sensible use of variables often means less coding

**<span style="color: red;">Some drawbacks:</span>**
* Requires learning new API methods – you'll still likely have to read documentation 
* Overly abstracted APIs can obscure underlying logic (Py2Neo avoids this)
* Abstracted database querying logic can lead to inefficient code

### Connecting to Neo4j with the Py2Neo Middleware

In [453]:
# Let's First Delete Our Previous Database Entries (Nodes & Relationships)
session.run("MATCH (n) OPTIONAL MATCH (n)-[r]-() DELETE n, r")

# Import the Py2Neo Middleware
from py2neo import Node, Relationship, Graph, NodeSelector

# Set Some References
database = "http://localhost:7474/db/data/"
username = "neo4j"
password = "test"

# Connect to Neo4j- Note the default username/password is neo4j/neo4j
graph    = Graph(database, user = username, password = password)

#### Creating & Querying Nodes

In [454]:
# List of Legendary Characters' Properties
characterProps = [
    {"fullName": 'Artorius Pendragon',   "title": 'King',    "age": 37,  "influence": 1.00, "lovesSpam": True},
    {"fullName": 'Gwythyr ap Greidawl',  "title": 'Queen',   "age": 32,  "influence": 0.75, "lovesSpam": True},
    {"fullName": 'Lancelot du Lac',      "title": 'Knight',  "age": 29,  "influence": 0.25, "lovesSpam": False},
    {"fullName": 'Myrddin Wyllt',        "title": 'Wizard',  "age": 68,  "influence": 0.50, "lovesSpam": True}
]

storedNodes   = []

# Generate Graph Nodes
for properties in characterProps:
    node      = Node("Person", **properties)
    n         = graph.create(node)
    storedNodes.append(n)

# Select Graph Nodes
selector      = NodeSelector(graph)
nodes         = selector.select("Person")

# Print Another Silly Haiku
print("Another Legendary (kinda) Haiku of Yore\n")
for node in nodes:
    properties = dict(node)
    if properties["lovesSpam"]:
        print("%s was a legendary %s of spam" % (properties["fullName"], properties["title"].lower()))
    else:
        print("Little %s was a legendary %s of the spam rebel alliance" % (properties["fullName"], properties["title"].lower()))
print("\n-------------------------------------------------------------------------------------------------------------")

# We Can Also Select by Property or Conditionally Using the Where Function
print("Let these highly accurate not at all fabricated historical facts be known...\n")
nodes       = selector.select("Person").where("_.influence > 0.72", "_.lovesSpam = True")
for node in nodes:
    properties = dict(node)
    print("%s was an influential %s who survived on a diet highly enriched with spam." % (properties["fullName"], properties["title"]))
    

Another Legendary (kinda) Haiku of Yore

Artorius Pendragon was a legendary king of spam
Gwythyr ap Greidawl was a legendary queen of spam
Little Lancelot du Lac was a legendary knight of the spam rebel alliance
Myrddin Wyllt was a legendary wizard of spam

-------------------------------------------------------------------------------------------------------------
Let these highly accurate not at all fabricated historical facts be known...

Artorius Pendragon was an influential King who survived on a diet highly enriched with spam.
Gwythyr ap Greidawl was an influential Queen who survived on a diet highly enriched with spam.


#### Creating & Querying Edges

In [455]:
# Set This to a Relationship of Interest
queryRelationship = "FRIEND_TO"

# Generating a Name Map – Because We Didn't Give Short Names to Each Node
nameMap = {
    "Arthur"    : 'Artorius Pendragon',
    "Guinevere" : 'Gwythyr ap Greidawl',
    "Lancelot"  : 'Lancelot du Lac',
    "Merlin"    : 'Myrddin Wyllt'
}

# Relationships Dictionary
relationships = [
    {"a": "Arthur",    "type": "HUSBAND_TO", "b": "Guinevere"},
    {"a": "Arthur",    "type": "KING_TO",    "b": "Lancelot"},
    {"a": "Arthur",    "type": "KING_TO",    "b": "Merlin"},
    {"a": "Arthur",    "type": "ADVISEE_TO", "b": "Merlin"},
    {"a": "Guinevere", "type": "WIFE_TO",    "b": "Arthur"},
    {"a": "Guinevere", "type": "FRIEND_TO",  "b": "Lancelot"},
    {"a": "Guinevere", "type": "FRIEND_TO",  "b": "Merlin"},
    {"a": "Lancelot",  "type": "SUBJECT_TO", "b": "Arthur"},
    {"a": "Lancelot",  "type": "FRIEND_TO",  "b": "Guinevere"},
    {"a": "Merlin",    "type": "ADVISOR_TO", "b": "Arthur"},
    {"a": "Merlin",    "type": "FRIEND_TO",  "b": "Guinevere"}
]

# Initialize Selector
selector    = NodeSelector(gdb)
nodes       = selector.select("Person")

# Generate Relationships
for relationship in relationships:
    nodeA = list(selector.select("Person", fullName = nameMap[relationship["a"]]))[0]
    nodeB = list(selector.select("Person", fullName = nameMap[relationship["b"]]))[0]
    result = gdb.create(Relationship(nodeA, relationship["type"], nodeB))

# Query Relationships
relationships = graph.match(rel_type = queryRelationship)
    
# Print Relationship
for relationship in relationships:
    print(relationship)

(a1680a5)-[:FRIEND_TO]->(cae1b25)
(b1eff88)-[:FRIEND_TO]->(cae1b25)
(cae1b25)-[:FRIEND_TO]->(b1eff88)
(cae1b25)-[:FRIEND_TO]->(a1680a5)


It's worth noting that the relationship printed above prints the IDs for each node in the relationship rather than the named IDs as we saw previously. This is because the a unique ID is automatically generated using Py2Neo. While this may make the output less legible, it has the added benefit of ensuring of preventing duplicate IDs that may otherwise slip through. 

# Data Access

We've placed the full [Disease Module Identification DREAM Challenge](https://www.synapse.org/#!Synapse:syn6156761/wiki/400645) data along with a small subset for this tutorial in a folder on the OHSU Church server. We'll be using the subset for the rest of the tutorial, but if you're feeling bold, go ahead and grab the full data and modify the code to load up a complete network. To do this:

* Open up your preferred terminal app
* If you haven't already cloned this repository to your local machine, go ahead and do that now
    * To see how to do this, see the README.md of the Github repository
* With the repository cloned, cd to its location.

### Just a Subset Please (< 10 MB uncompressed)

* Make a data directory and cd
```
mkdir -p data/subchallenge1/ && cd data/subchallenge1/
```
* Pull the data from Church: 
```
scp USERNAME@church.ohsu.edu:~/../cordierb/neo4j-tutorial/data/subchallenge1/2_ppi_anonym_v2.txt .
```

### Full Disease Module Detection DREAM Challenge Data (~300 - 400MB uncompressed)

* Pull the complete data folder from Church: 
```
scp -r USERNAME@church.ohsu.edu:~/../cordierb/neo4j-tutorial/data .
```

**<span style="color: green;">Tip:</span>** *If you're unfamiliar with the scp command, in brief, it allows you to transfer files and directories between machines (usually your local machine and a server). The trailing '.' in the command will pull the data to your current directory – if you're not already in the repository's directory, you can specify an absolute path in place of the '.' (e.g. ~/Desktop/neo4j-tutorial)*

# Exploring The DREAM Data With NetworkX

While Neo4j is a cool NoSQL database, given that the DREAM data for this tutorial has a relatively small footprint and Neo4j has few builtin tools for graph analysis, a more sane approach is to just load up a DREAM TSV file into NetworkX and start an analysis, which is the approach we've opted for below. 
 
### Importing the DREAM data

In [457]:
import csv as CSV

# Set to Data Directory
dataFile = "data/subchallenge1/1_ppi_anonym_v2.txt"

# Open Data and Store edgeList Reference
with open(dataFile, "r") as data:
    edgeData = CSV.reader(data, delimiter = "\t")
    # The Edge Data is Imported as Strings, Let's Type the Edge Weights to Floats and Nodes to Integers
    edgeList = [tuple(map(lambda x: float(x) if "." in x else int(x), edge)) for edge in edgeData]
    
# So What Does an Individual Edge Look Like?
print("Node A \tNode B \tEdge Weight")
print("\t".join(str(x) for x in edgeList[0]))

Node A 	Node B 	Edge Weight
0	3	0.461


### Generating The Network

### Creating an Edge

### Writing a Query

### Module Detection

# Common Issues & Their Solutions

### If You Previously Installed Neo4j and Forgot Your Username/Password

Resetting your Neo4j password isn't exactly intuitive. If you installed Neo4j using Homebrew previously and forgot your password, here's a reset method that *should* work (admittedly, I haven't quite sorted out the intricacies of this methods' behavior, but it has worked for me before and won't do any harm...): 

* Navigate to the location of Neo4j's authentication file (again, replace `VERSION` with the version number you just wrote down):
```
cd ~/../../usr/local/Cellar/neo4j/VERSION/libexec/data/dbms/
```
* Next, remove the authentication file and restart Neo4j.
```
rm auth
neo4j restart
```
* In your browser, go to [`http://localhost:7474`](http:////localhost:7474).

If it worked, you should be prompted to setup your authentication (again). 

**<span style="color: green;">Tip:</span>** This method was tested on a Homebrew installation, but if you can find the root of your Neo4j installation, from there you should be able to access ```data/dbms/``` and perform ```rm auth``` or similar.*

### Port Bind Exception

You may encounter the following error (piled within a big, ugly stack trace):
```
org.neo4j.helpers.PortBindException: Address localhost:7687 is already in use, cannot bind to it
```

The likely culprit is another java program running on your OS. A potential solution for this is to terminate all Java processes and then restarting neo4j:
```
killall -9 java
neo4j console
```

### Server Certificate Does Not Match Known Certificate for 'localhost'

You may encounter the following error when using the Python Neo4j driver middleware after upgrading Neo4j:
```
ProtocolError: Server certificate does not match known certificate for 'localhost'; check details in file '/Users/Benjamin/.neo4j/known_hosts'
```

If you're not accessing Neo4j using multiple hosts (which is unlikely), open up a terminal tab and run the following commands:
```
cd ~/.neo4j
rm known_hosts
touch known_hosts
```

If you are using multiple hosts, follow the above but, instead of removing the known_hosts file, edit the file to remove the localhost entry and its associated certificate key.

### General SSL Engine Problem

This error is the Java version of the above error:
```
General SSLEngine problem
```

To solve this, follow the same steps for the above error (Server Certificate Does Not Match Known Certificate for 'localhost').